# ReadMe

# Prep

In [8]:
import json
import sqlite3
from pathlib import Path
import pandas as pd
import os

In [4]:
data_dir = Path("Data")
data_dir.mkdir(exist_ok=True)  # ensure the data folder exists

csv_path = data_dir / "supply_chain_sales.csv"
db_path = data_dir / "mini_datalake.db"

In [5]:
conn = sqlite3.connect(db_path)

In [2]:
df=pd.read_csv('../Data/supply_chain_sales.csv')
print(df.shape)
df.head()

(60000, 12)


,date,manufacturer,distributor,retailer,product,quantity,base_price,sale_price,GMV,COGS,profit,discount_percent
0,2025-09-01,MF_3,DS_6,RT_44,Product_2,52,300000,242957.14,12633771.36,10354616.05,2279155.32,19.01
1,2025-09-01,MF_1,DS_8,RT_32,Product_5,61,180000,158513.39,9669317.05,7094859.70,2574457.35,11.94
2,2025-09-01,MF_2,DS_4,RT_24,Product_1,87,150000,148257.49,12898401.77,11090727.44,1807674.33,1.16
3,2025-09-01,MF_3,DS_9,RT_19,Product_1,24,150000,130473.35,3131360.30,1931809.67,1199550.63,13.02
4,2025-09-01,MF_2,DS_10,RT_49,Product_4,88,250000,208377.87,18337252.38,12170466.19,6166786.19,16.65


In [6]:
df.to_sql("sales_data", conn, if_exists="replace", index=False)
print(f"💾 Saved table 'sales_data' inside {db_path}")

💾 Saved table 'sales_data' inside Data\mini_datalake.db


In [7]:
check = pd.read_sql("SELECT COUNT(*) AS total_rows FROM sales_data", conn)
display(check)

,total_rows
0,60000


# Gemini API

In [9]:
# Load API key from config.json or env var
KEY = None
cfg_path = Path("config.json")
if cfg_path.exists():
    KEY = json.loads(cfg_path.read_text()).get("GEMINI_API_KEY")
else:
    KEY = os.getenv("GEMINI_API_KEY")

In [10]:
def init_gemini(api_key):
    configure(api_key=api_key)
    # choose the model you have access to (e.g., "gemini-1.5-pro")
    return GenerativeModel("gemini-2.5-flash")

In [1]:
from google.generativeai import configure, GenerativeModel

ModuleNotFoundError: No module named 'google.generativeai'

In [12]:
FEW_SHOT_EXAMPLES = [
    {
        "sql_result": "region | total_gmv\nNorth  | 12500000\nSouth  | 9800000\n",
        "summary": "North leads sales with 12.5M GMV; recommend reviewing stock levels in South where GMV is 9.8M."
    },
    {
        "sql_result": "retailer | total_gmv\nRT_5 | 3,000,000\nRT_12| 2,700,000\n",
        "summary": "Top retailer RT_5 generated 3.0M GMV; consider offering targeted promotions to RT_12 to close the gap."
    }
]


In [ ]:
def build_prompt(sql_result_text, user_question="Please summarize top insights and give 2 action items."):
    few_shot_text = ""
    for ex in FEW_SHOT_EXAMPLES:
        few_shot_text += f"Example SQL result:\n{ex['sql_result']}\nDesired summary: {ex['summary']}\n\n---\n\n"

    prompt = f"""
You are a concise business analyst. Given SQL query results, write a short plain-English executive summary (2-3 sentences), followed by 2 short, practical action items the operations team can take. Stick to facts and avoid speculation.

{few_shot_text}

Now here is the SQL result to analyze:
{sql_result_text}

User query: {user_question}

Provide:
1) Executive summary
2) Action item 1
3) Action item 2
"""
    return prompt

# Example usage (initialize only if KEY present)
gemini = None
if KEY:
    gemini = init_gemini(KEY)
else:
    print("Gemini not initialized (missing key).")